In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
#pd.set_option('max_rows',999)
pd.options.display.float_format = '{:.9f}'.format
import random
import itertools
from pathlib import Path
import re
import datetime
import dateutil
from multiprocessing import Pool
import matplotlib.pyplot as plt
from sklearn import linear_model
#import networkx as nx

from data_transform import file_to_transform, df_trades_resample, trades_files, trades2_transformed_files
#root = Path("/home/daniele/Documenti/Progetti/TimeSeries/borsa/download_data")
from load_dfs import load_dfs, intersez_date, date_limite, max_intersection
#print(f"Root directory: {root}")

In [ ]:
def make_pd_date_interval(inizio, fine, frequenza):
    future = pd.date_range(inizio,fine, freq=frequenza).strftime("%Y-%b-%d").tolist()
    future = pd.DataFrame(future)
    future.columns = ['ds']
    future['ds']= pd.to_datetime(future['ds'])
    return future

def make_pd_date_interval_train_test(df, train_perc, freq='D'):
    daterange = pd.date_range(df['ds'][0], df['ds'][-1], freq=freq)#.strftime("%Y-%b-%d").tolist()
    interval = daterange[-1] - daterange[0]
    train_interval = pd.Timedelta(int(interval.days*train_perc), "d")
    
    # train
    end_training =  df['ds'][0] + train_interval
    train = pd.date_range(df['ds'][0], end_training, freq=freq)#.strftime("%Y-%b-%d").tolist()
    train = pd.DataFrame(train)
    train.columns = ['ds']
    train['ds']= pd.to_datetime(train['ds'])
    
    #test
    future = pd.date_range(end_training, df['ds'][-1], freq=freq)#.strftime("%Y-%b-%d").tolist()
    future = pd.DataFrame(future)
    future.columns = ['ds']
    future['ds']= pd.to_datetime(future['ds'])
    
    return train, future

In [ ]:
resampled_data_folder = Path("resampled")

## Gestione Dati

### Carico da file i dataframe ricampionati

In [ ]:
df_list, coinpairs = load_dfs(resampled_data_folder, "df5min")

In [ ]:
df_list[99]#.plot()

## Overlap massimo

In [ ]:
# range_tot = list(range(len(df_list)))
# coppie_numeric = list(itertools.combinations(range_tot, 2))

In [ ]:
# inters = [intersez_date(df_list, i,j) for i, j in coppie_numeric]
# print(max(inters))
# best_ij = coppie_numeric[np.argmax(inters)]
# best = date_limite(df_list, best_ij[0], best_ij[1])
# print(best)

In [ ]:
# lista_best_idx = []

# try:
#     range_tot.remove(best_ij[0])
#     range_tot.remove(best_ij[1])
# except:
#     pass

In [ ]:
# lunghezze = []
# while len(range_tot) > 1:
#     inters_both = [max_intersection(df_list, best, i) for i in range_tot]
#     inters = [a-b for a,b in inters_both]
#     lungh = max(inters)
#     #print(lungh)
#     lunghezze.append(lungh)
#     best_i = np.argmax(inters)
#     best = max_intersection(df_list, best, range_tot[best_i])
#     lista_best_idx.append(range_tot[best_i])
#     range_tot.pop(best_i)

In [ ]:
# plt.plot([l.days for l in lunghezze])

### Scelgo un numero di crypto per massimizzare la finestra temporale

In [ ]:
# cut = 150
# dfs = pd.concat([df_list[i] for i in lista_best_idx[:cut]], join='inner', axis=1)

In [ ]:
#dfs = dfs2.copy()

In [ ]:
# devo riempire i vuoti che ci sono negli intervalli privi di transazioni, ricopio il prezzo precedente
#dfs.fillna(0, inplace=True)
# dfs = (dfs.ffill()+dfs.bfill())/2
# for col in dfs.columns:
#     dfs[col] = dfs[col].replace(to_replace=0, method='ffill')

In [ ]:
# coinpairs = [c.split('-')[0] for c in coinpairs]
# dfs.columns = [coinpairs[i] for i in lista_best_idx[:cut]]

In [ ]:
# coinpairs.index("BTC_EUR")

In [ ]:
# dfs['BTC_EUR'].plot()

In [ ]:
# dfs['BTC_EUR'].shape

## Prendo solo il bitcoin

In [ ]:
df = df_list[99]
df_1day = df.resample('D').mean()
df_1day = df_1day.reset_index(level=0)
df_1day.columns = ['ds', 'y']

In [ ]:
df_1day['ds_'] = df_1day['ds']
df_1day = df_1day.set_index('ds_')
#df_1day

In [ ]:
#make_pd_date_interval('2021-06-10','2021-07-10', 'D')
train, test = make_pd_date_interval_train_test(df_1day, 0.56)

In [ ]:
df_train = pd.merge(train, df_1day, on='ds')
df_test = pd.merge(test, df_1day, on='ds')

In [ ]:
from prophet import Prophet
model = Prophet(daily_seasonality=True)

model.fit(df_train)

In [ ]:
#forecast = model.predict(test)
#forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
#forecast.plot()

In [ ]:
#model.plot_components(forecast)


In [ ]:
forecast = model.predict(df_test)

In [ ]:
model.plot(forecast)
df_1day.y.plot()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
y_true = df_test.y
y_pred = forecast['yhat'].values
mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)
print(f'MAE: {round(mae,3)} \t MAPE: {round(mape,5)}')

In [ ]:
100-(0.04981*100)

In [ ]:
100-(100*abs(12-11.5)/12)

In [ ]:
2500/2900

## Cross validation

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(model, initial='365 days', period='30 days', horizon = '365 days')


In [ ]:
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')
